<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

[https://en.wikipedia.org/wiki/List_of_Falcon\_9\_and_Falcon_Heavy_launches](https://en.wikipedia.org/wiki/List_of_Falcon\_9\_and_Falcon_Heavy_launches?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01)


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_1\_L2/images/Falcon9\_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing\_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_1\_L2/images/falcon9-launches-wiki.png)


## Objectives

Web scrap Falcon 9 launch records with `BeautifulSoup`:

*   Extract a Falcon 9 launch records HTML table from Wikipedia
*   Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [5]:
# use requests.get() method with the provided static_url
# assign the response to a object
RocketData = requests.get(static_url).text

Create a `BeautifulSoup` object from the HTML `response`


In [6]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(RocketData,"html5lib")

Print the page title to verify if the `BeautifulSoup` object was created properly


In [7]:
# Use soup.title attribute
print(soup.title)

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [8]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables = soup.find_all('table') 

In [9]:
#html_tables

Starting from the third table is our target table contains the actual launch records.


In [10]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
#print(first_launch_table)

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [11]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names


In [12]:
columns = soup.find_all('th')

for name in columns:
    column_names.append(name.text.strip())

Check the extracted column names


In [13]:
#print(column_names)

In [14]:
launch_dict = dict.fromkeys(column_names)

In [15]:
#launch_dict

In [16]:
# Remove an irrelvant column
del launch_dict['Date and time (UTC)']

In [17]:
#launch_dict

## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [18]:
# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [19]:
#launch_dict

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [20]:
len(launch_dict)

198

In [21]:
#launch_dict

In [22]:
#Extract each table 
extracted_row = 0
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            

In [23]:
#print(launch_dict.values())

In [24]:
#flight_number = 999
#launch_dict['Flight No.'].append(flight_number)
#launch_dict

In [25]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            flight_number
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            launch_dict['Flight No.'].append(flight_number)
            
            #print(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            #print(date)
            launch_dict['Date'].append(date)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            #print(time)
            launch_dict['Time'].append(time)            
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            #print(bv)
            launch_dict['Version Booster'].append(bv)            
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            #print(launch_site)
            launch_dict['Launch site'].append(launch_site)            
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            #print(payload)
            launch_dict['Payload'].append(payload)            
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            #print(payload)
            launch_dict['Payload mass'].append(payload_mass)            
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            #print(orbit)
            launch_dict['Orbit'].append(orbit)   
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            #print(launch_outcome)
            launch_dict['Launch outcome'].append(launch_outcome)            

            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            #print(booster_landing)
            launch_dict['Booster landing'].append(booster_landing)            
            
            
#            # Customer
#            # TODO: Append the customer into launch_dict with key `Customer`
#            customer = row[6].a.string
#            #print(customer)
#            launch_dict['Customer'].append(customer)            

            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            try:
               customer = row[6].a.string
#              print(customer)
               launch_dict['Customer'].append(customer)            
            except:
               launch_dict['Customer'].append("999")   
                  

In [26]:
#launch_dict

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [27]:
df=pd.DataFrame(launch_dict)

In [28]:
#df.head

In [29]:
df.columns

Index(['Flight No.', 'Date andtime (UTC)', 'Version,Booster [b]',
       'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer',
       'Launchoutcome', 'Boosterlanding',
       ...
       'Launches by rocket type', 'Launches by spaceport',
       'Agencies, companiesand facilities', 'Other mission listsand timelines',
       'Payload', 'Launch outcome', 'Version Booster', 'Booster landing',
       'Date', 'Time'],
      dtype='object', length=198)

In [33]:
df.shape

(121, 198)

In [34]:
df[df['Launch site'] == 'CCAFS']

,Flight No.,Date andtime (UTC),"Version,Booster [b]",Launch site,Payload[c],Payload mass,Orbit,Customer,Launchoutcome,Boosterlanding,...,Launches by rocket type,Launches by spaceport,"Agencies, companiesand facilities",Other mission listsand timelines,Payload,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,None,None,CCAFS,None,0,LEO,SpaceX,None,None,...,None,None,None,None,Dragon Spacecraft Qualification Unit,Success\n,F9 v1.0B0003.1,Failure,4 June 2010,18:45
1,2,None,None,CCAFS,None,0,LEO,NASA,None,None,...,None,None,None,None,Dragon,Success,F9 v1.0B0004.1,Failure,8 December 2010,15:43
2,3,None,None,CCAFS,None,525 kg,LEO,NASA,None,None,...,None,None,None,None,Dragon,Success,F9 v1.0B0005.1,No attempt\n,22 May 2012,07:44
3,4,None,None,CCAFS,None,"4,700 kg",LEO,NASA,None,None,...,None,None,None,None,SpaceX CRS-1,Success\n,F9 v1.0B0006.1,No attempt,8 October 2012,00:35
4,5,None,None,CCAFS,None,"4,877 kg",LEO,NASA,None,None,...,None,None,None,None,SpaceX CRS-2,Success\n,F9 v1.0B0007.1,No attempt\n,1 March 2013,15:10
6,7,None,None,CCAFS,None,"3,170 kg",GTO,SES,None,None,...,None,None,None,None,SES-8,Success,F9 v1.1,No attempt,3 December 2013,22:41
7,8,None,None,CCAFS,None,"3,325 kg",GTO,Thaicom,None,None,...,None,None,None,None,Thaicom 6,Success,F9 v1.1,No attempt,6 January 2014,22:06
46,47,None,None,CCAFS,None,C,LEO,Northrop Grumman,None,None,...,None,None,None,None,Zuma,Success,F9 B4,Success,8 January 2018,01:00
47,48,None,None,CCAFS,None,"4,230 kg",GTO,SES,None,None,...,None,None,None,None,GovSat-1,Success,F9 FTB1032.2,Controlled,31 January 2018,21:25
49,50,None,None,CCAFS,None,"6,092 kg",GTO,Hispasat,None,None,...,None,None,None,None,Hispasat 30W-6,Success,F9 B4,No attempt,6 March 2018,05:33


In [35]:
df['Launch site'].value_counts()

CCAFS             40
KSC               33
Cape Canaveral    20
VAFB              16
CCSFS             12
Name: Launch site, dtype: int64

In [36]:
df.loc[((df['Launch site'] == 'CCAFS') & (df['Booster landing'] == "Success\n"))]

,Flight No.,Date andtime (UTC),"Version,Booster [b]",Launch site,Payload[c],Payload mass,Orbit,Customer,Launchoutcome,Boosterlanding,...,Launches by rocket type,Launches by spaceport,"Agencies, companiesand facilities",Other mission listsand timelines,Payload,Launch outcome,Version Booster,Booster landing,Date,Time


In [37]:
df['Orbit'].value_counts()

LEO            67
GTO            33
SSO             7
Polar           7
MEO             3
HEO             2
Sub-orbital     1
Polar orbit     1
Name: Orbit, dtype: int64

In [38]:
df[df['Launch outcome'] == 'none']

,Flight No.,Date andtime (UTC),"Version,Booster [b]",Launch site,Payload[c],Payload mass,Orbit,Customer,Launchoutcome,Boosterlanding,...,Launches by rocket type,Launches by spaceport,"Agencies, companiesand facilities",Other mission listsand timelines,Payload,Launch outcome,Version Booster,Booster landing,Date,Time


In [40]:
df[df['Launch outcome'] == 'Success\n']

,Flight No.,Date andtime (UTC),"Version,Booster [b]",Launch site,Payload[c],Payload mass,Orbit,Customer,Launchoutcome,Boosterlanding,...,Launches by rocket type,Launches by spaceport,"Agencies, companiesand facilities",Other mission listsand timelines,Payload,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,None,None,CCAFS,None,0,LEO,SpaceX,None,None,...,None,None,None,None,Dragon Spacecraft Qualification Unit,Success\n,F9 v1.0B0003.1,Failure,4 June 2010,18:45
3,4,None,None,CCAFS,None,"4,700 kg",LEO,NASA,None,None,...,None,None,None,None,SpaceX CRS-1,Success\n,F9 v1.0B0006.1,No attempt,8 October 2012,00:35
4,5,None,None,CCAFS,None,"4,877 kg",LEO,NASA,None,None,...,None,None,None,None,SpaceX CRS-2,Success\n,F9 v1.0B0007.1,No attempt\n,1 March 2013,15:10
8,9,None,None,Cape Canaveral,None,"2,296 kg",LEO,NASA,None,None,...,None,None,None,None,SpaceX CRS-3,Success\n,F9 v1.1,Controlled,18 April 2014,19:25
14,15,None,None,Cape Canaveral,None,570 kg,HEO,USAF,None,None,...,None,None,None,None,DSCOVR,Success\n,F9 v1.1,Controlled,11 February 2015,23:03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,117,None,None,CCSFS,None,"15,600 kg",LEO,SpaceX,None,None,...,None,None,None,None,Starlink,Success\n,F9 B5B1051.10,Success,9 May 2021,06:42
117,118,None,None,KSC,None,"~14,000 kg",LEO,SpaceX,None,None,...,None,None,None,None,Starlink,Success\n,F9 B5B1058.8,Success,15 May 2021,22:56
118,119,None,None,CCSFS,None,"15,600 kg",LEO,SpaceX,None,None,...,None,None,None,None,Starlink,Success\n,F9 B5B1063.2,Success,26 May 2021,18:59
119,120,None,None,KSC,None,"3,328 kg",LEO,NASA,None,None,...,None,None,None,None,SpaceX CRS-22,Success\n,F9 B5B1067.1,Success,3 June 2021,17:29


In [41]:
df[df['Launch outcome'] == 'Success']

,Flight No.,Date andtime (UTC),"Version,Booster [b]",Launch site,Payload[c],Payload mass,Orbit,Customer,Launchoutcome,Boosterlanding,...,Launches by rocket type,Launches by spaceport,"Agencies, companiesand facilities",Other mission listsand timelines,Payload,Launch outcome,Version Booster,Booster landing,Date,Time
1,2,None,None,CCAFS,None,0,LEO,NASA,None,None,...,None,None,None,None,Dragon,Success,F9 v1.0B0004.1,Failure,8 December 2010,15:43
2,3,None,None,CCAFS,None,525 kg,LEO,NASA,None,None,...,None,None,None,None,Dragon,Success,F9 v1.0B0005.1,No attempt\n,22 May 2012,07:44
5,6,None,None,VAFB,None,500 kg,Polar orbit,MDA,None,None,...,None,None,None,None,CASSIOPE,Success,F9 v1.1B1003,Uncontrolled,29 September 2013,16:00
6,7,None,None,CCAFS,None,"3,170 kg",GTO,SES,None,None,...,None,None,None,None,SES-8,Success,F9 v1.1,No attempt,3 December 2013,22:41
7,8,None,None,CCAFS,None,"3,325 kg",GTO,Thaicom,None,None,...,None,None,None,None,Thaicom 6,Success,F9 v1.1,No attempt,6 January 2014,22:06
9,10,None,None,Cape Canaveral,None,"1,316 kg",LEO,Orbcomm,None,None,...,None,None,None,None,Orbcomm-OG2,Success,F9 v1.1,Controlled,14 July 2014,15:15
10,11,None,None,Cape Canaveral,None,"4,535 kg",GTO,AsiaSat,None,None,...,None,None,None,None,AsiaSat 8,Success,F9 v1.1,No attempt,5 August 2014,08:00
11,12,None,None,Cape Canaveral,None,"4,428 kg",GTO,AsiaSat,None,None,...,None,None,None,None,AsiaSat 6,Success,F9 v1.1,No attempt\n,7 September 2014,05:00
12,13,None,None,Cape Canaveral,None,"2,216 kg",LEO,NASA,None,None,...,None,None,None,None,SpaceX CRS-4,Success,F9 v1.1,Uncontrolled,21 September 2014,05:52
13,14,None,None,Cape Canaveral,None,"2,395 kg",LEO,NASA,None,None,...,None,None,None,None,SpaceX CRS-5,Success,F9 v1.1,Failure,10 January 2015,09:47


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab.

Following labs will be using a provided dataset to make each lab independent.


In [42]:
df.to_csv('c:\spacex_web_scraped.csv', index=False)

## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">Nayef Abou Tayoun</a>


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description          |
| ----------------- | ------- | ---------- | --------------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates               |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.


In [44]:
df=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_1.csv")

In [46]:
#df.head

In [47]:
df['Orbit'].value_counts()

GTO      27
ISS      21
VLEO     14
PO        9
LEO       7
SSO       5
MEO       3
GEO       1
ES-L1     1
HEO       1
SO        1
Name: Orbit, dtype: int64

In [48]:
df['BoosterVersion'].value_counts()

Falcon 9    90
Name: BoosterVersion, dtype: int64

In [60]:
df['LandingPad'].isnull().sum()

26

In [58]:
df[21:40].isnull()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
21,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
22,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
23,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
24,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
25,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
26,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
27,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
28,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
29,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
30,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False


In [68]:
df['Outcome'].value_counts()

True ASDS      41
None None      19
True RTLS      14
False ASDS      6
True Ocean      5
False Ocean     2
None ASDS       2
False RTLS      1
Name: Outcome, dtype: int64

In [66]:
df.columns

Index(['FlightNumber', 'Date', 'BoosterVersion', 'PayloadMass', 'Orbit',
       'LaunchSite', 'Outcome', 'Flights', 'GridFins', 'Reused', 'Legs',
       'LandingPad', 'Block', 'ReusedCount', 'Serial', 'Longitude',
       'Latitude'],
      dtype='object')